In [1]:
### Load preprocessed data

In [ ]:
#!python ../src/download_ml20.py

In [5]:
import numpy as np
fh = np.load('data/dataset_ml20_wide.npz')
# We have a bunch of feature columns and last column is the y-target
train_items = fh['train_items'].astype(np.int64)
train_ratng = fh['train_ratng'].astype(np.int64)
test_items = fh['test_items'].astype(np.int64)
test_ratng = fh['test_ratng'].astype(np.int64)

n_user = train_items.shape[0]
n_rank = train_items.shape[1]
n_item = train_items.max() + 1

In [7]:
!pip install -q reformer_pytorch

In [9]:
import random
import torch
import numpy as np
import pandas as pd
from torch import nn
from torch import from_numpy
import pytorch_lightning as pl
from torch.nn import functional as F
from reformer_pytorch import Reformer
from pytorch_lightning.callbacks import ModelCheckpoint

In [ ]:
class AR(pl.LightningModule):
    def __init__(self, n_item, n_dim, n_resp, heads=6, depth=6):
        self.n_dim = n_dim
        self.n_item = n_item
        self.n_resp = n_resp
        self.item = nn.Embedding(n_items, n_dim)
        self.resp = nn.Embedding(n_responses, n_dim)
        self.reformer = Reformer(dim=n_dim, depth=depth, heads=heads, causal=True)
        self.user_lin = nn.Linear(n_dim, n_dim * n_responses)
        self.item_lin = nn.Linear(n_dim, n_dim * n_responses)
    
    def forward(self, items, ratng):
        item_vec = self.item(items)
        resp_vec = self.resp(ratng)
        intx_vec = item_vec * resp_vec
        user_vec = self.reformer(intx_vec)
        return user_vec
    
    def likelihood(self, user_vec, items, ratng):
        batchsize = user_vec.shape[0]
        item_vec = self.item(items)
        # Broadcast (batchsize, n_dim) to (batchsize, n_dim, n_resp)
        user_vec_resp = self.user_lin(user_vec).reshape((batchsize, self.n_dim, self.n_resp))
        item_vec_resp = self.item_lin(user_vec).reshape((batchsize, self.n_dim, self.n_resp))
        # Sum interactions back down to  (batchsize, n_resp)
        intx = (user_vec_resp * item_vec_resp).sum(dim=1)
        loss = F.softmax(intx, ratng)